ALL THE EXTERNAL REUIQREMENTS ARE PRESNET IN ./requirement.txt

In [1]:
%pip install -r requirement.txt

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 239.6/239.6 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 54.9/54.9 kB 950.7 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 708.0/708.0 kB 1.6 MB/s eta 0:00:00
     ------------------------------------ 196.5/196.5 kB 995.9 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproje

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1075 lines of output]
      Dependence list: ['pybind11<2.6.2', 'psutil', "numpy>=1.10.0,<1.17 ; python_version=='2.7'", "numpy>=1.10.0 ; python_version>='3.5'"]
      c:\ProgramData\anaconda3\lib\site-packages\setuptools\dist.py:770: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
        warnings.warn(
      c:\ProgramData\anaconda3\lib\site-packages\setuptools\installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
      running bdist_wheel
      running build
      running build_ext
      Extra compilation arguments: ['/EHsc', '/openmp', '/O2', '/DVERSION_INFO=\\"2.1.1\\"']
      building 'nmslib' extension
      creating build
      creating build\t

In [2]:
#error can occur, careful!!
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.0.0/en_core_web_md-3.0.0.tar.gz
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 47.0/47.0 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 11.1/11.1 MB 11.1 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 10.8 MB/s eta 0:00:00
     -------------------------------------- 126.0/126.0 kB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 82.8/82.8 kB 4.8 MB/s eta 0:00:00
  Created wheel for en-core-web-md: filename=en_core_web_md-3.0.0-py3-none-any.whl size=47053321 sha256=fe6775624d6c750af9d3ac20a99bf51758516afe3527bc876a12eed64f85d85f
  Stored in directory: c:\users\prath\appdata\local\pip\cache\wheels\75\47\b6\1138aff8b49cd20e39c338e51d2ae35d79fb86e77c7ad97c60
Successfully built en-core-web-md
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstal

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.0.9 which is incompatible.
flask 2.2.2 requires click>=8.0, but you have click 7.1.2 which is incompatible.
black 22.6.0 requires click>=8.0.0, but you have click 7.1.2 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 125.1/125.1 MB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.4.0-py3-none-any.whl size=125666871 sha256=8f23452baa3fc5cbffec5c7f2593d4fcce03b0d8c6bca7276b1af50c88658a8f
  Stored in directory: c:\users\prath\appdata\local\pip\cache\wheels\c3\f5\32\313d08b812c91abeb6fb1d3b0f8fd69687c30c3a9d38288e4c
Successfully built en-ner-bc5cdr-md
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
cwd_path= os.getcwd()
cwd_path= cwd_path.replace("\\","/")

Lets scrape the required dataset 
Input will be a text file containing articles abstracts from pubmed

In [4]:
from my_packages.get_document import module_get_document

kg_disease= str(input("please enter the disease you want a knowledge graph for: "))
no_of_articles= int(input("please enter the number of articles you want to consider: ")) #expected wall time for 10 articles around 12 mins
dataset_file_path= cwd_path + "/datasets/dataset.txt"

module_get_document.get_docx(kg_disease, no_of_articles, dataset_file_path)


Search results saved to a text file.


Lets crete a log file for ouputs 

In [5]:
log_file_path= cwd_path + "/ouput/output_log.txt"

try:
    with open(log_file_path, 'w') as fh:
        fh.write("welcome to the log file!")
except Exception as e:
    print(e)

Lets create a dataframe that can be later deployed to neo4J

In [6]:
from my_packages.get_knowledgeGraph import module_getKnowledgeGrpah

output_df_path= module_getKnowledgeGrpah.get_gph(cwd_path, log_file_path, dataset_file_path)


100%|██████████| 122/122 [04:04<00:00,  2.00s/it]


Lets deploy df to neo4J :)

In [7]:
from my_packages.create_graph import module_createGraph
#the load_gh fn takes input of bolt url and password of sandbox, create your own sandbow on neo4J
class_labels= module_createGraph.load_gh("bolt://44.202.64.28:7687", "cams-profit-sacks", output_df_path)

{'total': 118, 'batches': 1, 'time': 4.788581371307373}
{'total': 0, 'batches': 1, 'time': 3.8949801921844482}
{'total': 0, 'batches': 1, 'time': 1.6946802139282227}
{'total': 0, 'batches': 1, 'time': 0.9032599925994873}


* Ignore below if you dont need to check efficiency!!

**calculating the efficiency of 5 abastract indivially
- manually abstracting entites from each abstact
- creating a list of list
- chcking acciracy, no of enties found, no of entites not found
- returning a dataframe

In [2]:
entities_list = [
    # Abstract 1
    [
        "Non-Hodgkin lymphoma",
        "organ",
        "body",
        "symptoms",
        "physicians",
        "specialties",
        "biology",
        "genetics",
        "diagnostic methods",
        "therapies",
        "The Lancet"
    ],
    
    # Abstract 2
    [
        "Lymphomas",
        "Hodgkin",
        "Non-Hodgkin",
        "organ",
        "B cell",
        "T cell",
        "natural killer cell",
        "indolent lymphomas",
        "aggressive lymphomas",
        "chemosensitive"
    ],

    # Abstract 3
    [
        "Hodgkin lymphoma",
        "Non-Hodgkin lymphoma",
        "malignant lymphomas",
        "lymphocytes",
        "clonal proliferation",
        "natural history",
        "epidemiology",
        "pathologic characteristics",
        "clinical presentation",
        "optimal management",
        "The Lancet"
    ],

    # Abstract 4
    [
        "T-cell lymphomas",
        "Hodgkin lymphomas",
        "histiocytic/dendritic cell neoplasms",
        "World Health Organization (WHO) system",
        "pathological",
        "genetic",
        "clinical factors",
        "T-follicular helper cell-derived lymphomas",
        "indolent T-cell lymphomas",
        "lymphoproliferative disorders"
    ],

    # Abstract 5
    [
        "lymphomas",
        "normal lymphoid system",
        "B-cell",
        "T-cell",
        "natural killer-cell neoplasms",
        "multiparameter approach",
        "Revised European and American Lymphoma",
        "WHO classifications",
        "molecular alterations",
        "The Lancet"
    ]
]

In [1]:
dataset= [
    "Lymphomas can affect any organ in the body, present with a wide range of symptoms, and be seen by primary care physicians and physicians from most specialties. They are traditionally divided into Hodgkin's lymphoma (which accounts for about 10% of all lymphomas) and non-Hodgkin lymphoma, which is the topic of this Seminar. Non-Hodgkin lymphoma represents a wide spectrum of illnesses that vary from the most indolent to the most aggressive malignancies. They arise from lymphocytes that are at various stages of development, and the characteristics of the specific lymphoma subtype reflect those of the cell from which they originated. Since this topic was last reviewed in The Lancet in 2012, advances in understanding the biology and genetics of non-Hodgkin lymphoma and the availability of new diagnostic methods and therapies have improved our ability to manage patients with this disorder.",

    "Lymphomas may be broadly divided into non-Hodgkin (90%) and Hodgkin (10%) types. Most lymphomas (90%) are of B cell origin but can also be T cell or natural killer cell. Clinical management of indolent and aggressive lymphomas is different. Aggressive lymphomas are more dangerous if left untreated yet a higher cell proliferation rate also renders them more chemosensitive, so they are managed with curative intent. Indolent lymphomas are, for the most part, incurable, such that quality of life must be balanced against toxicity of treatment in deciding when and how to treat."

    "The malignant lymphomas, including both Hodgkin lymphoma (HL) and non-Hodgkin lymphoma (NHL), represent a diverse group of diseases that arise from a clonal proliferation of lymphocytes. Each of the more than 30 unique types of lymphoma is a disease with a distinct natural history. This biologic heterogeneity gives rise to marked differences among the lymphomas with respect to epidemiology, pathologic characteristics, clinical presentation, and optimal management. This article emphasizes the principles of diagnosis, including appropriate pathologic evaluation and staging considerations, and focuses on the clinical presentation, staging, and optimal management strategies for the most common types of lymphoma.",

    "Lymphomas are classified based on the normal counterpart, or cell of origin, from which they arise. Because lymphocytes have physiologic immune functions that vary both by lineage and by stage of differentiation, the classification of lymphomas arising from these normal lymphoid populations is complex. Recent genomic data have contributed additional depth to this complexity. Areas covered: Lymphoma classification follows the World Health Organization (WHO) system, which reflects international consensus and is based on pathological, genetic, and clinical factors. The present review focuses on the classification of T-cell lymphomas, Hodgkin lymphomas, and histiocytic and dendritic cell neoplasms, summarizing changes reflected in the 2016 revision to the WHO classification. These changes are critical to hematologists and other clinicians who care for patients with these disorders. Expert commentary: Lymphoma classification is a continually evolving field that needs to be responsive to new clinical, pathological, and molecular understanding of lymphoid neoplasia. Among the entities covered in this review, the 2016 revisions in the WHO classification particularly impact T-cell lymphomas, including a new umbrella category of T-follicular helper cell-derived lymphomas and evolving recognition of indolent T-cell lymphomas and lymphoproliferative disorders.",

    "Our current understanding of the normal lymphoid system informs the modern classification of lymphomas. B-cell, T-cell, and natural killer-cell neoplasms often recapitulate normal stages of lymphoid cell differentiation and function. Moreover, the clinical manifestations of lymphomas often reflect the normal function of lymphoid cells in vivo. The multiparameter approach to classification adopted by the Revised European and American Lymphoma and subsequent WHO classifications facilitates the interpretation of clinical and translational studies, and provides a framework for the discovery of molecular alterations that drive these tumors. An accurate and precise classification of disease entities facilitates the discovery of the molecular basis of lymphoid neoplasms in the basic science laboratory, and leads to new diagnostic tools that play a role in clinical diagnosis.",
]

In [3]:
from my_packages.check_efficiencyGraph import module_get_efficiency
df= []
for i in range(4):
    accuracy= module_get_efficiency.check_ef(dataset[i], entities_list[i])
    df.append([i, accuracy])

In [4]:
df

[[0, 0.07142857142857142], [1, 0.13333333333333333], [2, 0.0], [3, 0.0]]